<a href="https://colab.research.google.com/github/charleszhang418/SpaceX/blob/main/code/dnabert_finetune_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

+ Data: https://osdr.nasa.gov/bio/repo/data/studies/OSD-466
+ Sample:
  1. RR10_FCS_FLT_KO_F19: p21-null, Space Flight
  2. RR10_FCS_FLT_WT_F16: Wild Type, Space Flight
  3. RR10_FCS_GC_WT_G3: Wild Type, Ground Control
  4. RR10_FCS_GC_KO_G4: p21-null, Ground Control
  5. RR10_FCS_VIV_WT_V1: Wild Type, Vivarium Control
  6. RR10_FCS_VIV_KO_V13: p21-null,	Vivarium Control

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# %cd gdrive/MyDrive/NASA
%cd gdrive/MyDrive/Project/data_science/nasa-space-app-2023/
!ls

/content/gdrive/MyDrive/Project/data_science/nasa-space-app-2023
code  data  model


In [3]:
!pip install transformers
!pip install torch
!pip install einops
!pip install transformers[torch]
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, BertModel
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import evaluate
import numpy as np

In [13]:
dna_data = pd.read_csv('data/dna_data.csv')
print(dna_data.shape)
dna_data = dna_data.drop_duplicates(subset='DNA')
print(dna_data.shape)
dna_data.head()

(36000, 5)
(35867, 5)


,notation,DNA,mass,filename,label
0,A00654:48:HN52TDRXY:1:2101:5990:1000 2:N:0:AGT...,ATATTTATGGCTGGACTTGAACTTACTAAGTAGACCATGCTGGCCT...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type
1,A00654:48:HN52TDRXY:1:2101:6840:1000 2:N:0:AGT...,CCTACGCTCAGCGAGGCGACTTTGAGAGATGCGCCGAAGAATCTTT...,"FFFF:F,F:FFFFFFFFFFF,FFF::FFFFFFFFF,,F,,FFF,F:...",GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type
2,A00654:48:HN52TDRXY:1:2101:9498:1000 2:N:0:AGT...,GCCTTGACCCATGCCTGATAAGGGAGGGCCCGGTCGACGCCCAGGA...,":FFFFFFFFFFFFFFFF:FFFFFFFFFF,FF:FFFFF:FFFFFFFF...",GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type
3,A00654:48:HN52TDRXY:1:2101:15067:1000 2:N:0:AG...,GGACAGGGCCGCAGCATATTCTCATTAAACGGCTGGCCGTCATGGT...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type
4,A00654:48:HN52TDRXY:1:2101:1298:1016 2:N:0:AGT...,TCCTCCATCCATGCTTTGAATGTCTGGAACCCTGCCTCTGTGTTAC...,FFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFF:F...,GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type


In [14]:
# Label proportion
from collections import Counter
my_list = list(dna_data['label'])
element_count = Counter(my_list)
total_elements = len(my_list)
element_percentages = {key: (count / total_elements) * 100 for key, count in element_count.items()}
for key, percentage in element_percentages.items():
    print(f"{key}: {percentage:.2f}%")

Wild Type: 49.97%
p21-null: 50.03%


In [6]:
tokenizer = AutoTokenizer.from_pretrained('zhihan1996/DNABERT-2-117M', trust_remote_code=True)
# model = BertModel.from_pretrained('zhihan1996/DNABERT-2-117M', trust_remote_code=True)
model = BertForSequenceClassification.from_pretrained('zhihan1996/DNABERT-2-117M', num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.encoder.layer.5.attention.self.value.weight', 'bert.encoder.layer.11.attention.self.value.weight', 'bert.encoder.layer.5.intermediate.dense.bias', 'bert.pooler.dense.weight', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.1.intermediate.dense.bias', 'bert.encoder.layer.9.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.3.output.dense.bias', 'bert.encoder.layer.4.attention.self.query.weight', 'bert.encoder.layer.3.attention.self.value.weight', 'bert.encoder.layer.5.attention.self.query.bias', 'bert.encoder.layer.3.attention.self.query.weight', 'bert.encoder.layer.1.attention.self.key.bias', 'bert.encoder.layer.11.attention.self.query.weight', 'bert.encoder.layer.8.output.dense.bias', 'bert.encoder.layer.2.output

In [7]:
# # Create random label for testing
# import random
# N = len(dna_data)
# random_label_list = [random.randint(0, 2) for _ in range(N)]
# dna_data['label'] = random_label_list

In [8]:
# Label encoding

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(list(dna_data['label']))
dna_data['label'] = encoded_labels

# original_labels = label_encoder.inverse_transform(encoded_labels)

In [9]:
train_data = dna_data.sample(frac=0.8, random_state=524)
test_eval_data = dna_data.drop(train_data.index)
eval_data = test_eval_data.sample(frac=0.5, random_state=524)
test_data = test_eval_data.drop(eval_data.index)

print(train_data.shape, eval_data.shape, test_data.shape)

train_dna = list(train_data['DNA'])
train_labels = list(train_data['label'])

val_dna = list(eval_data['DNA'])
val_labels = list(eval_data['label'])

test_dna = list(test_data['DNA'])
test_labels = list(test_data['label'])

train_encodings = tokenizer(train_dna, truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(val_dna, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_dna, truncation=True, padding=True, return_tensors='pt')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(28800, 5) (3600, 5) (3600, 5)


In [10]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [11]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  acc = metric.compute(predictions=predictions, references=labels)['accuracy']
  return {
      'accuracy': acc
  }

training_args = TrainingArguments(
    output_dir='model/results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    learning_rate=1e-5,
    logging_dir='model/logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(test_dataset)

# model.save_pretrained("./your_finetuned_model")
# tokenizer.save_pretrained("./your_finetuned_tokenizer")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.705000,0.687610,0.554444
2,0.687100,0.691607,0.546111
3,0.674400,0.683279,0.550278
4,0.661700,0.664781,0.563611
5,0.648300,0.680356,0.550278
6,0.640700,0.686674,0.544722
7,0.634300,0.662543,0.556111
8,0.626200,0.665035,0.558056
9,0.621000,0.684837,0.559167
10,0.618100,0.684570,0.560833


In [12]:
results

{'eval_loss': 0.6819718480110168,
 'eval_accuracy': 0.5719444444444445,
 'eval_runtime': 3.192,
 'eval_samples_per_second': 1127.832,
 'eval_steps_per_second': 35.401,
 'epoch': 10.0}